# Basic Model Interface (BMI) for streamflow routing using t-route

This t-route network was developed for use in the Next Generation National Water Model (Nextgen). Nextgen runs models with Basic Model Interface (BMI).

In [1]:
sys.path.append("src/")
import bmi_troute # This is the BMI t-route that we will be running from the file: bmi_troute.py 
import troute.main_utilities as mu # This is used to read q_lateral data from files, won't be needed when t-route bmi is run with model engine
import troute.nhd_preprocess as nhd_prep # This is used to read q_lateral data from files, won't be needed when t-route bmi is run with model engine

In [2]:
model = bmi_troute.bmi_troute()

In [3]:
model.initialize(bmi_cfg_file='test/BMI/test_AnA_bmi.yaml')

creating supernetwork connections set
supernetwork connections set complete
setting waterbody and channel initial states ...
waterbody and channel initial states complete


In [4]:
(run_sets, da_sets, parity_sets) = mu.build_run_sets(model._network,
                                                     model._forcing_parameters,
                                                     model._compute_parameters, 
                                                     model._data_assimilation_parameters,
                                                     model._output_parameters,
                                                     model._parity_parameters)

q_lateral, coastal_boundary_depth_df = nhd_prep.nhd_forcing(run_sets[0], 
                                                            model._forcing_parameters, 
                                                            model._hybrid_parameters,
                                                            model._network.segment_index,
                                                            model._compute_parameters.get('cpu_pool', None),
                                                            model._network._t0,             
                                                            model._network._coastal_boundary_depth_df,
        )

data_assimilation = mu.build_data_assimilation(model._network,
                                               model._data_assimilation_parameters,
                                               model._waterbody_parameters,
                                               [], #da_sets[0],
                                               model._forcing_parameters,
                                               model._compute_parameters)

In [5]:
model.set_value('land_surface_water_source__volume_flow_rate', q_lateral.to_numpy())
#model.set_value('coastal_boundary__depth', coastal_boundary_depth_df.to_numpy())
#model.set_value('usgs_gage_observation__volume_flow_rate', data_assimilation.reservoir_usgs_df.to_numpy())
#model.set_value('usace_gage_observation__volume_flow_rate', src)
#model.set_value('rfc_gage_observation__volume_flow_rate', src)

In [6]:
model.update()

In [7]:
model.get_value('channel_exit_water_x-section__volume_flow_rate')

1611674      0.000091
1611676      0.000085
3763126      0.034257
3762962      0.002928
3762964      0.001314
               ...   
942080042    0.303696
942080043    0.763048
942080044    3.949975
942080045    1.003813
942090009    0.022306
Name: (11, q), Length: 10806, dtype: float32

In [8]:
model.get_current_time()

3600.0

Now utilize the update_until() function to run t-route for only a portion of the total time

In [9]:
model.initialize(bmi_cfg_file='test/BMI/test_AnA_bmi.yaml')

creating supernetwork connections set
supernetwork connections set complete
setting waterbody and channel initial states ...
waterbody and channel initial states complete


In [10]:
model.set_value('land_surface_water_source__volume_flow_rate', q_lateral.to_numpy())
#model.set_value('coastal_boundary__depth', coastal_boundary_depth_df.to_numpy())
#model.set_value('usgs_gage_observation__volume_flow_rate', src)
#model.set_value('usace_gage_observation__volume_flow_rate', src)
#model.set_value('rfc_gage_observation__volume_flow_rate', src)

In [11]:
model.update_until(1800) # input value is seconds. 1800s = 0.5 hours

In [12]:
model.get_current_time()

1800.0

In [13]:
model.get_value('channel_exit_water_x-section__volume_flow_rate')

1611674      0.000084
1611676      0.000083
3763126      0.034257
3762962      0.002928
3762964      0.001314
               ...   
942080042    0.303720
942080043    0.763046
942080044    3.948903
942080045    1.003609
942090009    0.022577
Name: (5, q), Length: 10806, dtype: float32

Now call update() to run t-route for the remainder of the full model run

In [14]:
model.update()

In [15]:
model.get_current_time()

3600.0

In [16]:
model.get_value('channel_exit_water_x-section__volume_flow_rate')

1611674      0.000091
1611676      0.000085
3763126      0.034257
3762962      0.002928
3762964      0.001314
               ...   
942080042    0.303696
942080043    0.763048
942080044    3.949975
942080045    1.003813
942090009    0.022306
Name: (5, q), Length: 10806, dtype: float32

Finalize the model by deleting all internal objects

In [ ]:
model.finalize()